In [1]:
!pip install requests pandas numpy shapely scikit-learn tqdm

In [2]:
import requests
import pandas as pd
import numpy as np
import math
import time
import os
from tqdm import tqdm
from sklearn.neighbors import BallTree

# Put your keys here
WAQI_TOKEN = "4c0735999b909ffa3580a1ffea0044acb2b671f1"
OPENWEATHER_API_KEY = "b9ccc55124c700e32d7baf7f4e0b74ac"

# Files
GRID_FILE = "locations_25km_india.csv"
FINAL_OUTPUT = "module1_waqi_weather_osm_india.csv"

# Settings
SLEEP = 0.15
OSM_RADIUS_M = 500
WEATHER_ROUND_PRECISION = 1

In [3]:
df_grid = pd.read_csv(GRID_FILE)
print("Grid points:", len(df_grid))
df_grid.head()

Grid points: 5056


,id,lat,lon
0,pt_00001,8.093018,77.443716
1,pt_00002,8.317596,77.222009
2,pt_00003,8.317596,77.448974
3,pt_00004,8.317596,77.675939
4,pt_00005,8.317596,77.902904


In [4]:
def get_waqi_pollution(lat, lon):
    url = f"https://api.waqi.info/feed/geo:{lat};{lon}/"
    params = {"token": WAQI_TOKEN}

    try:
        r = requests.get(url, params=params, timeout=20)
        r.raise_for_status()
        data = r.json()
        
        if data.get("status") != "ok":
            return None
        
        iaqi = data["data"].get("iaqi", {})
        
        return {
            "aqi": data["data"].get("aqi"),
            "pm25": iaqi.get("pm25", {}).get("v"),
            "pm10": iaqi.get("pm10", {}).get("v"),
            "no2": iaqi.get("no2", {}).get("v"),
            "so2": iaqi.get("so2", {}).get("v"),
            "co": iaqi.get("co", {}).get("v"),
            "o3": iaqi.get("o3", {}).get("v")
        }
    except Exception as e:
        return None

In [7]:
weather_cache = {}

def get_weather(lat, lon):
    key = (round(lat, WEATHER_ROUND_PRECISION), round(lon, WEATHER_ROUND_PRECISION))
    if key in weather_cache:
        return weather_cache[key]

    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"lat": lat, "lon": lon, "appid": OPENWEATHER_API_KEY, "units": "metric"}

    try:
        r = requests.get(url, params=params, timeout=20)
        r.raise_for_status()
        d = r.json()
        result = {
            "temp_c": d.get("main", {}).get("temp"),
            "humidity": d.get("main", {}).get("humidity"),
            "wind_speed": d.get("wind", {}).get("speed"),
            "wind_deg": d.get("wind", {}).get("deg"),
            "weather_desc": d.get("weather", [{}])[0].get("description")
        }
    except:
        result = {
            "temp_c": None, "humidity": None,
            "wind_speed": None, "wind_deg": None,
            "weather_desc": None
        }

    weather_cache[key] = result
    return result


In [9]:
weather_cache = {}

def get_weather(lat, lon):
    key = (round(lat, WEATHER_ROUND_PRECISION), round(lon, WEATHER_ROUND_PRECISION))
    if key in weather_cache:
        return weather_cache[key]

    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"lat": lat, "lon": lon, "appid": OPENWEATHER_API_KEY, "units": "metric"}

    try:
        r = requests.get(url, params=params, timeout=20)
        r.raise_for_status()
        d = r.json()
        result = {
            "temp_c": d.get("main", {}).get("temp"),
            "humidity": d.get("main", {}).get("humidity"),
            "wind_speed": d.get("wind", {}).get("speed"),
            "wind_deg": d.get("wind", {}).get("deg"),
            "weather_desc": d.get("weather", [{}])[0].get("description")
        }
    except:
        result = {
            "temp_c": None, "humidity": None,
            "wind_speed": None, "wind_deg": None,
            "weather_desc": None
        }

    weather_cache[key] = result
    return result

In [11]:
osm_cache = {}

def get_osm(lat, lon, radius=OSM_RADIUS_M):
    key = (round(lat,2), round(lon,2))
    if key in osm_cache:
        return osm_cache[key]

    lat_min = lat - (radius/111111)
    lat_max = lat + (radius/111111)
    lon_factor = radius/(111111 * max(0.0001, abs(math.cos(math.radians(lat)))))
    lon_min = lon - lon_factor
    lon_max = lon + lon_factor

    bbox = f"{lat_min},{lon_min},{lat_max},{lon_max}"

    query = f"""
    [out:json][timeout:25];
    (
      way["highway"]({bbox});
      way["landuse"="industrial"]({bbox});
      way["landuse"="farmland"]({bbox});
      node["amenity"="waste_disposal"]({bbox});
      node["landuse"="landfill"]({bbox});
    );
    out body;
    """

    try:
        r = requests.post("https://overpass-api.de/api/interpreter", data=query, timeout=30)
        data = r.json()
    except:
        data = {"elements": []}

    features = {"roads": 0, "industrial": 0, "farmland": 0, "dump": 0}

    for el in data.get("elements", []):
        tags = el.get("tags", {}) or {}
        if "highway" in tags:
            features["roads"] += 1
        if tags.get("landuse") == "industrial":
            features["industrial"] += 1
        if tags.get("landuse") == "farmland":
            features["farmland"] += 1
        if tags.get("amenity") == "waste_disposal" or tags.get("landuse") == "landfill":
            features["dump"] += 1

    osm_cache[key] = features
    return features

In [13]:
CHECKPOINT_FILE = "module1_checkpoint.csv"

records = []

# --------------------------
# RESUME LOGIC
# --------------------------
if os.path.exists(CHECKPOINT_FILE):
    df_prev = pd.read_csv(CHECKPOINT_FILE)
    processed_ids = set(df_prev["id"])
    records = df_prev.to_dict("records")
    print(f"Resuming from checkpoint — {len(processed_ids)} rows already completed.")
else:
    processed_ids = set()
    records = []
    print("No checkpoint found — starting fresh.")


# --------------------------
# MAIN LOOP WITH CHECKPOINTS
# --------------------------
for i, r in tqdm(df_grid.iterrows(), total=len(df_grid), desc="Processing grid"):
    lat = r["lat"]
    lon = r["lon"]
    row_id = r["id"]

    # Skip if already processed
    if row_id in processed_ids:
        continue

    # ----------------------
    # Pollution (WAQI)
    # ----------------------
    p = get_waqi_pollution(lat, lon) or {
        "aqi": None, "pm25": None, "pm10": None,
        "no2": None, "so2": None, "co": None, "o3": None
    }

    # ----------------------
    # Weather (cached)
    # ----------------------
    w = get_weather(lat, lon)

    # ----------------------
    # OSM land features (cached)
    # ----------------------
    o = get_osm(lat, lon)

    # Merge into one row
    rec = {
        "id": row_id,
        "lat": lat,
        "lon": lon,
        **p,
        **w,
        **o
    }
    records.append(rec)
    processed_ids.add(row_id)

    # ----------------------
    # CHECKPOINT every 30 rows
    # ----------------------
    if i % 30 == 0 and i > 0:
        pd.DataFrame(records).to_csv(CHECKPOINT_FILE, index=False)
        print(f"Checkpoint saved at row {i} — total saved: {len(records)}")

    time.sleep(SLEEP)


# --------------------------
# FINAL SAVE
# --------------------------
df_final = pd.DataFrame(records)
df_final.to_csv(FINAL_OUTPUT, index=False)
print("Processing complete!")
print("Saved final dataset:", FINAL_OUTPUT)

Resuming from checkpoint — 1261 rows already completed.


Processing grid:  26%|███████████████                                            | 1290/5056 [02:23<1:27:02,  1.39s/it]

Checkpoint saved at row 1290 — total saved: 1291


Processing grid:  26%|███████████████▍                                           | 1321/5056 [04:25<3:09:34,  3.05s/it]

Checkpoint saved at row 1320 — total saved: 1321


...

Checkpoint saved at row 5010 — total saved: 5011


Processing grid: 100%|██████████████████████████████████████████████████████████▊| 5041/5056 [2:36:31<00:02,  5.66it/s]

Checkpoint saved at row 5040 — total saved: 5041


Processing grid: 100%|███████████████████████████████████████████████████████████| 5056/5056 [2:36:34<00:00,  1.86s/it]


Processing complete!
Saved final dataset: module1_waqi_weather_osm_india.csv


In [15]:
df_final.to_csv(FINAL_OUTPUT, index=False)
print("Saved dataset:", FINAL_OUTPUT)
df_final.head()

Saved dataset: module1_waqi_weather_osm_india.csv


,id,lat,lon,aqi,pm25,pm10,no2,so2,co,o3,temp_c,humidity,wind_speed,wind_deg,weather_desc,roads,industrial,farmland,dump
0,pt_00001,8.093018,77.443716,93.0,93.0,63.0,0.1,7.0,3.1,6.7,28.66,59.0,3.56,51.0,overcast clouds,3,0,0,0
1,pt_00002,8.317596,77.222009,93.0,93.0,63.0,0.1,7.0,3.1,6.7,32.26,43.0,2.06,210.0,overcast clouds,34,0,0,0
2,pt_00003,8.317596,77.448974,93.0,93.0,63.0,0.1,7.0,3.1,6.7,30.75,45.0,2.43,46.0,overcast clouds,6,0,0,0
3,pt_00004,8.317596,77.675939,66.0,NaN,66.0,3.9,2.5,7.3,11.8,28.75,56.0,6.60,14.0,overcast clouds,5,0,0,0
4,pt_00005,8.317596,77.902904,66.0,NaN,66.0,3.9,2.5,7.3,11.8,28.37,62.0,7.83,20.0,overcast clouds,3,0,0,0
